<a href="https://colab.research.google.com/github/JerrrrryL/cifar-100-effNet/blob/master/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/davda54/sam.git
! pip install opacus
! git clone https://github.com/hyhmia/BlindMI.git

fatal: destination path 'sam' already exists and is not an empty directory.
fatal: destination path 'BlindMI' already exists and is not an empty directory.


In [2]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet50', 'resnet101']


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.GroupNorm(32, planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.GroupNorm(32, planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.GroupNorm(32, planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

        gn_init(self.bn1)
        gn_init(self.bn2)
        gn_init(self.bn3, zero_init=True)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


def conv2d_init(m):
    assert isinstance(m, nn.Conv2d)
    n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
    m.weight.data.normal_(0, math.sqrt(2. / n))

def gn_init(m, zero_init=False):
    assert isinstance(m, nn.GroupNorm)
    m.weight.data.fill_(0. if zero_init else 1.)
    m.bias.data.zero_()


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.GroupNorm(32, 64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                conv2d_init(m)
        gn_init(self.bn1)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(32, planes * block.expansion),
            )
            m = downsample[1]
            assert isinstance(m, nn.GroupNorm)
            gn_init(m)

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet50(**kwargs):
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model

def resnet101(**kwargs):
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model

In [3]:
import torch

import sys
import torch.nn as nn
sys.path.append('sam/example/')
from utility.cutout import Cutout
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from utility.log import Log
from utility.initialize import initialize
from utility.step_lr import StepLR
from opacus import PrivacyEngine
import opacus.utils.module_modification as module_modification
from opacus.dp_model_inspector import DPModelInspector
from torchvision.models import wide_resnet101_2

class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm


In [4]:
import torch
import os
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import numpy as np
import torchvision
import torchvision.transforms as transforms
from PIL.Image import BICUBIC
from opacus.utils.uniform_sampler import UniformWithReplacementSampler
import torchvision.models as models

dropout = 0.2
learning_rate = 8e-4
label_smoothing = 0.001
momentum = 0.9
threads = 2
rho = 0.05
weight_decay = 0.0005
noise_multiplier = 0.005
DELTA = 1e-5
class_num = 100
MODEL_PATH = 'target_models/'
DATASET = 'cifar-100/'
if not os.path.exists(MODEL_PATH + DATASET):
      os.makedirs(MODEL_PATH + DATASET)

# def get_statistics(class_num):
#         if class_num == 10:
#             train_set = torchvision.datasets.CIFAR10(root='./cifar', train=True, download=True,
#                                                      transform=transforms.ToTensor())
#             test_set = torchvision.datasets.CIFAR10(root='./cifar', train=False, download=True,
#                                                     transform=transforms.ToTensor())
#         else:
#             train_set = torchvision.datasets.CIFAR100(root='./cifar', train=True, download=True,
#                                                       transform=transforms.ToTensor())
#             test_set = torchvision.datasets.CIFAR100(root='./cifar', train=False, download=True,
#                                                      transform=transforms.ToTensor())

#         data = torch.cat([d[0] for d in DataLoader(train_set)] + [d[0] for d in DataLoader(test_set)])
#         return data.mean(dim=[0, 2, 3]), data.std(dim=[0, 2, 3])

In [5]:
def split_dataset(dataset, num_splits, batch_size):
  train_size = len(dataset) // num_splits
  sample_rate = batch_size / train_size
  split_datasets = []
  x_remaining = dataset
  for i in range(num_splits-1):
    x_train, x_remaining = torch.utils.data.random_split(x_remaining, [train_size, len(x_remaining)-train_size])
    split_datasets.append(torch.utils.data.DataLoader(x_train, 
                                                      batch_size=batch_size,
                                                      num_workers=threads))
  split_datasets.append(torch.utils.data.DataLoader(x_remaining, 
                                                    batch_size=batch_size,
                                                    num_workers=threads))
  return split_datasets

def train_target_model(train_set, test_set, run, class_num=100, learning_rate=0.1, batch_size=64, virtual_batch_size=256,
                       noise_multiplier = 0.002, max_grad_norm=10, epochs=15, dropout=0, weight_decay=0.0005, 
                       dp=False, pretrained=False, SGD=True):
  torch.cuda.empty_cache()
  n_acc_steps = virtual_batch_size / batch_size
  sample_rate = batch_size / len(train_set)
  if pretrained:
    model = None
    model = resnet50()
    state_dict = torch.load("ImageNet-ResNet50-GN.pth")["state_dict"]
    # create new OrderedDict that does not contain `module.`
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(dropout),
        nn.Linear(num_ftrs, class_num)
    )
  else:
    model = torchvision.models.AlexNet(num_classes=class_num)

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  # print(model)
  model = model.to(device)
  log = Log(log_each=10)

  privacy_engine = PrivacyEngine(
      model,
      sample_rate=sample_rate,
      alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)),
      noise_multiplier=noise_multiplier,
      max_grad_norm=max_grad_norm,
  )
  # DP-SGD
  if SGD:
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    if dp:
      privacy_engine.attach(optimizer)
  # SAM
  else:
    base_optimizer = torch.optim.SGD
    if dp:
      privacy_engine.attach(base_optimizer)
    optimizer = SAM(model.parameters(), base_optimizer, rho=rho, lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
  scheduler = StepLR(optimizer, learning_rate, epochs)
  for epoch in range(epochs):
      model.train()
      criterion = nn.CrossEntropyLoss()
      log.train(len_dataset=len(train_set))
      
      for batch_ind, (inputs, targets) in enumerate(train_set):
          inputs, targets = inputs.to(device), targets.to(device) 

          # # Uncomment the following line for SAM
          # # first forward-backward step
          if SGD:
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = criterion(predictions, targets)
            loss.backward()
            # SGD
            if dp:
              if ((batch_ind + 1) % n_acc_steps == 0) or ((batch_ind + 1) == len(train_set)):
                  optimizer.step()
              else:
                  # accumulate per-example gradients but don't take a step yet
                  optimizer.virtual_step()
            else:
              optimizer.step()
          else:
            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets)
            loss.mean().backward()

            # Uncomment the following for SAM
            optimizer.first_step(zero_grad=True)

            # second forward-backward step
            smooth_crossentropy(model(inputs), targets).mean().backward()
            optimizer.second_step(zero_grad=True)

          with torch.no_grad():
              correct = torch.argmax(predictions.data, 1) == targets
              # print(correct.size(0))
              # print(torch.cat(batch_size * [torch.reshape(loss.cpu(), (-1,))]))

              if SGD:
                log(model, 
                    torch.cat(batch_size * [torch.reshape(loss.cpu(), (-1,))]), 
                    correct.cpu(), 
                    scheduler.lr())
              else:
                log(model, loss.cpu(), correct.cpu(), scheduler.lr())
              scheduler(epoch)
      # rdp_sgd = get_renyi_divergence(privacy_engine.sample_rate, 
      #                                privacy_engine.noise_multiplier) * privacy_engine.steps
      # epsilon, _ = get_privacy_spent(rdp_sgd)
      # print(f"ε = {epsilon:.3f}")

      model.eval()
      log.eval(len_dataset=len(test_set))

      with torch.no_grad():
          for batch in test_set:
              inputs, targets = (b.to(device) for b in batch)

              predictions = model(inputs)
              loss = smooth_crossentropy(predictions, targets)
              correct = torch.argmax(predictions, 1) == targets
              log(model, loss.cpu(), correct.cpu())
      if dp:
          if not os.path.exists(MODEL_PATH + DATASET + 'dp/' + str(run)):
              os.makedirs(MODEL_PATH + DATASET + 'dp/' + str(run))
          torch.save(model, MODEL_PATH + DATASET + 'dp/' + str(run) + '/resnet_dp_0.005_' + str(epoch) + '.pt')
      elif not SGD:
          if not os.path.exists(MODEL_PATH + DATASET + 'sam/' + str(run)):
              os.makedirs(MODEL_PATH + DATASET + 'sam/' + str(run))
          torch.save(model, MODEL_PATH + DATASET + 'sam/' + str(run) + '/resnet_SAM_' + str(epoch) + '.pt')
      elif dropout != 0:
          if not os.path.exists(MODEL_PATH + DATASET + 'dropout/' + str(run)):
              os.makedirs(MODEL_PATH + DATASET + 'dropout/' + str(run))
          torch.save(model, MODEL_PATH + DATASET + 'dropout/' + str(run) + '/resnet_dropout_' + str(epoch) + '.pt')
      elif weight_decay != 0.0005:
          if not os.path.exists(MODEL_PATH + DATASET + 'l2_reg/' + str(run)):
              os.makedirs(MODEL_PATH + DATASET + 'l2_reg/' + str(run))
          torch.save(model, MODEL_PATH + DATASET + 'l2_reg/' + str(run) + '/resnet_l2_reg_' + str(epoch) + '.pt')
      else:
          if not os.path.exists(MODEL_PATH + DATASET + 'baseline/' + str(run)):
                os.makedirs(MODEL_PATH + DATASET + 'baseline/' + str(run))
          torch.save(model, MODEL_PATH + DATASET + 'baseline/' + str(run) + '/resnet_baseline_' + str(epoch) + '.pt')

  log.flush()
  return model

In [6]:
class AttackModel(nn.Module):
    def __init__(self):
        super(AttackModel, self).__init__()
        self.dout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(100, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 2)
        self.out = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dout(x)
        x = F.relu(self.fc2(x))
        x = self.dout(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.out(x)
        return x

def get_attack_data(shadow_model, input_train, input_test, validate=False):
  attack_x = None
  for batch in input_train:
    x, y = (b.to(device) for b in batch)
    # print(x.shape)
    pred = shadow_model(x).detach().cpu().numpy()
    if attack_x is None:
      attack_x = pred
    else:
      attack_x = np.r_[attack_x, pred]
  m_train = np.ones(len(attack_x))
  for batch in input_test:
    x, y = (b.to(device) for b in batch)
    # print(x.shape)
    pred = shadow_model(x).detach().cpu().numpy()
    if not validate:
      # we will augment the data so that it will not be biased
      attack_x = np.r_[attack_x, pred, pred, pred, pred, pred]
    else:
      attack_x = np.r_[attack_x, pred]
  m_test = np.zeros(len(attack_x) - len(m_train))
  return attack_x, np.r_[m_train, m_test]

def train_attack_model(shadow_train, shadow_test):
  # construct datasets for training attack models
  attack_x, attack_y = get_attack_data(shadow_model, shadow_train, shadow_test)
  attack_model = AttackModel()
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(attack_model.parameters(), lr=0.001, momentum=0.9)
  attack_x = torch.Tensor(attack_x) # transform to torch tensor
  attack_y = torch.Tensor(attack_y)
  attack_m = attack_y.type(torch.LongTensor)
  shokri_train = DataLoader(TensorDataset(attack_x, attack_m), shuffle=True) # create attack dataset

  for epoch in range(10):
      attack_model.train()
      for i, data in enumerate(shokri_train):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          # print(inputs[0], labels[0])

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = attack_model(inputs)
          # print(outputs, labels)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

  print('Finished Training')
  print('-'*10 + 'SUMMARY OF TRAIN ACC' + '*'*10)
  predict_membership = model_evaluation(attack_model, attack_x, attack_y)
  precision, recall, adversary_advantage = evaluation(predict_membership, attack_y.numpy())
  print("Precision: ", precision)
  print("Recall: ", recall)
  print("Adv Advantage: ", adversary_advantage)
  return attack_model

# evaluate the performance of the model
def model_evaluation(model, instances, membership):
  instances = torch.Tensor(instances)
  membership = torch.Tensor(membership)
  membership = membership.type(torch.LongTensor)
  dataloader = DataLoader(TensorDataset(instances, membership)) # create attack dataset
  predict_membership = []
  for i, data in enumerate(dataloader):
    input, target = data
    outputs = model(input)
    predict_membership += list(torch.argmax(outputs, 1).numpy())
  return predict_membership

# perform the neural network attack
def salem_membership_inference(target_model, shadow_train, shadow_test, validate_train, validate_test):
  # shadow_train/shadow_test are data for training the attack model
  # validate_train/validate_test are actual data to train the target model
  attack_model = train_attack_model(shadow_train, shadow_test)
  val_attack_x, val_attack_y = get_attack_data(target_model, validate_train, validate_test, validate=True)
  # evaluate accuracy on actual membership inference
  print('-'*10 + 'SUMMARY OF ATTACK' + '*'*10)
  predict_membership = model_evaluation(attack_model, val_attack_x, val_attack_y)
  precision, recall, adversary_advantage = evaluation(predict_membership, val_attack_y)
  print("Precision: ", precision)
  print("Recall: ", recall)
  print("Adv Advantage: ", adversary_advantage)

# return the precision, recall and adversary advantage
def evaluation(pred_membership, true_membership):
  tp, fp, tn, fn = 0, 0, 0, 0
  for i in range(len(true_membership)):
    if true_membership[i] == 1 and pred_membership[i] == 1:
      tp += 1
    elif true_membership[i] == 0 and pred_membership[i] == 1:
      fp += 1
    elif true_membership[i] == 1 and pred_membership[i] == 0:
      fn += 1
    else:
      tn += 1
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  adversary_advantage = tp / (tp + fn) - fp / (tn + fp)
  return precision, recall, adversary_advantage

# yeom's attack, return the precision, recall and adversary advantage
def yeom_attack(device, train, test, model):
    model = model.to(device)
    model.eval()
    train_correct = 0
    correct = 0
    total_train = 0
    total_test = 0
    total_loss = 0
    correct_prediction = 0
    yeom_prediction, per_instance_loss = [], []
    tp, fp, tn, fn = 0, 0, 0, 0
    with torch.no_grad():
        for batch in train:
            # compute the avg loss
            inputs, targets = (b.to(device) for b in batch)
            pred = model(inputs)
            loss = smooth_crossentropy(pred, targets)
            total_loss += loss.sum().item()
            total_train += targets.size(0)
        total_loss /= total_train
        # evaluate Yeom's attack
        for batch in train:
            # compute the avg loss
            inputs, targets = (b.to(device) for b in batch)
            pred = model(inputs)
            _, predicted = torch.max(pred, 1)
            loss = smooth_crossentropy(pred, targets)
            # append the per instance loss
            per_instance_loss += list(loss.detach().cpu().numpy())
            train_correct += (predicted == targets).sum().item()
            yeom_prediction += list((loss <= total_loss).detach().cpu().numpy() + 0)
        # print("This is the length", len(yeom_prediction))
        for batch in test:
            inputs, targets = (b.to(device) for b in batch)
            pred = model(inputs)
            _, predicted = torch.max(pred, 1)
            loss = smooth_crossentropy(pred, targets)
            # append the per instance loss
            per_instance_loss += list(loss.detach().cpu().numpy())
            total_test += targets.size(0)
            correct += (predicted == targets).sum().item()
            yeom_prediction += list((loss <= total_loss).detach().cpu().numpy() + 0)
        # print(inputs, targets)
    # print(per_instance_loss)
    yeom_prediction = np.array(yeom_prediction)
    per_instance_loss = np.array(per_instance_loss)
    tp, fn = np.sum(yeom_prediction[:total_train]), total_train - np.sum(yeom_prediction[:total_train])
    fp, tn = np.sum(yeom_prediction[total_train:]), total_test - np.sum(yeom_prediction[total_train:])
    print("-" * 10 + "Evaluation for Yeom's attack" + "-" * 10)
    print("The Test Accuracy is ", correct / total_test)
    print("True Positive: ", tp, "      True Negative: ", tn, "     False Positive: ", fp, "     False Negative: ", fn)
    print("Precision: ", tp / (tp + fp), "      Recall:", tp / (tp + fn))
    yeom_advantage = tp / (tp + fn) - fp / (tn + fp)
    print("Adversary Advantage: ", yeom_advantage)

    # print("-" * 10 + "Evaluation for Balanced Training and Testing sets" + "-" * 10)
    # random_index = np.random.choice(yeom_prediction[:total_test].shape[0], total_test, replace=False)
    # tp, fn = np.sum(yeom_prediction[random_index]), total_test - np.sum(yeom_prediction[random_index])
    # train_acc = train_correct / total_train
    # test_acc = correct / total_test
    # target_membership = np.concatenate([np.ones(total_train), np.zeros(total_test)])
    # yeom_advantage = tp / (tp + fn) - fp / (tn + fp)

    # print("The Test Accuracy is ", correct / total_test)
    # print("True Positive: ", tp, "      True Negative: ", tn, "     False Positive: ", fp, "     False Negative: ", fn)
    # print("Precision: ", tp / (tp + fp), "      Recall:", tp / (tp + fn))
    # res = [train_acc, test_acc, total_loss, target_membership, yeom_prediction, per_instance_loss, yeom_advantage]
    # return res

# the whole process, with attack models saved
# run is the index of run we are currently on
def run_experiment(run, save_data=False, num_splits=2, batch_size=32, virtual_batch_size=256, dp=False,
                   pretrained=True, learning_rate=0.1, SGD=True, dropout=0, l2_reg=0.0005):
  if save_data:
      mean, std = (0.5074, 0.4867, 0.4411), (0.2675, 0.2566, 0.2763)

      transform = transforms.Compose([
          transforms.Resize(224),
          transforms.ToTensor(),
          transforms.Normalize(mean, std)
      ])

      train_set = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
      test_set = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

      train_sets = split_dataset(train_set, num_splits, batch_size)
      test_sets = split_dataset(test_set, num_splits, batch_size)
      for i in range(num_splits):
        torch.save(train_sets[i], 'train' + str(i) + '.pt')
        torch.save(test_sets[i], 'test' + str(i) + '.pt')
  else:
      train_x = torch.load('train0.pt')
      test_x = torch.load('test0.pt')
      model = train_target_model(train_x, test_x, run, pretrained=pretrained, dp=dp, SGD=SGD,
                                 dropout=dropout, learning_rate=learning_rate, weight_decay=l2_reg)
      if dp:
          torch.save(model, MODEL_PATH + DATASET + 'resnet_dp_0.005.pt')
      elif SAM:
          torch.save(model, MODEL_PATH + DATASET + 'resnet_SAM.pt')
      elif dropout != 0:
          torch.save(model, MODEL_PATH + DATASET + 'resnet_dropout.pt')
      elif reg != 0.0005:
          torch.save(model, MODEL_PATH + DATASET + 'resnet_l2_reg.pt')
      else:
          torch.save(model, MODEL_PATH + DATASET + 'resnet_baseline.pt')
  return

In [7]:
# run_experiment(save_data=True)

In [8]:
# run experiment to generate target models
for i in range(1):
  run_experiment(i+1, learning_rate=8e-3)
  run_experiment(i+1, dropout=0.5, learning_rate=8e-3)
  run_experiment(i+1, l2_reg=0.0025, learning_rate=8e-3)
  run_experiment(i+1, SGD=False, learning_rate=8e-3)
  run_experiment(i+1, dp=True, learning_rate=8e-3)

┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:523: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:195: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


┃           0  ┃      1.8583  │     49.93 %  ┃   8.000e-03  │   02:15 min  ┃      1.3823  │     60.44 %  ┃
┃           1  ┃      1.0800  │     67.62 %  ┃   8.000e-03  │   02:15 min  ┃      1.2910  │     65.42 %  ┃
┃           2  ┃      0.8214  │     74.88 %  ┃   8.000e-03  │   02:15 min  ┃      1.4744  │     64.86 %  ┃
┃           3  ┃      0.6467  │     79.78 %  ┃   8.000e-03  │   02:15 min  ┃      1.4890  │     65.06 %  ┃
┃           4  ┃      0.5304  │     83.39 %  ┃   8.000e-03  │   02:15 min  ┃      1.4633  │     68.10 %  ┃
┃           5  ┃      0.1995  │     93.84 %  ┃   1.600e-03  │   02:15 min  ┃      1.2603  │     79.60 %  ┃
┃           6  ┃      0.0852  │     97.80 %  ┃   1.600e-03  │   02:15 min  ┃      1.3152  │     80.30 %  ┃
┃           7  ┃      0.0480  │     99.11 %  ┃   1.600e-03  │   02:15 min  ┃      1.3509  │     80.40 %  ┃
┃           8  ┃      0.0312  │     99.52 %  ┃   1.600e-03  │   02:15 min  ┃      1.3784  │     80.12 %  ┃
┃           9  ┃      0.0227  │     9

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:523: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:195: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


┃           0  ┃      2.2509  │     40.95 %  ┃   8.000e-03  │   02:15 min  ┃      1.3831  │     58.74 %  ┃
┃           1  ┃      1.3992  │     59.91 %  ┃   8.000e-03  │   02:15 min  ┃      1.3084  │     62.12 %  ┃
┃           2  ┃      1.1155  │     67.14 %  ┃   8.000e-03  │   02:15 min  ┃      1.3169  │     64.98 %  ┃
┃           3  ┃      0.9434  │     71.78 %  ┃   8.000e-03  │   02:15 min  ┃      1.3471  │     66.10 %  ┃
┃           4  ┃      0.8159  │     75.06 %  ┃   8.000e-03  │   02:15 min  ┃      1.5484  │     63.18 %  ┃
┃           5  ┃      0.3898  │     87.53 %  ┃   1.600e-03  │   02:15 min  ┃      1.2343  │     77.88 %  ┃
┃           6  ┃      0.2411  │     92.16 %  ┃   1.600e-03  │   02:15 min  ┃      1.3339  │     78.24 %  ┃
┃           7  ┃      0.1700  │     94.67 %  ┃   1.600e-03  │   02:15 min  ┃      1.3774  │     78.40 %  ┃
┃           8  ┃      0.1232  │     96.22 %  ┃   1.600e-03  │   02:15 min  ┃      1.4207  │     78.88 %  ┃
┃           9  ┃      0.0864  │     9

...

┃           0  ┃      1.8970  │     48.62 %  ┃   8.000e-03  │   02:15 min  ┃      1.3680  │     60.10 %  ┃
┃           1  ┃      1.1377  │     66.25 %  ┃   8.000e-03  │   02:15 min  ┃      1.2711  │     64.68 %  ┃
┃           2  ┃      0.9506  │     71.37 %  ┃   8.000e-03  │   02:15 min  ┃      1.3703  │     64.50 %  ┃
┃           3  ┃      0.8388  │     74.49 %  ┃   8.000e-03  │   02:15 min  ┃      1.3546  │     65.36 %  ┃
┃           4  ┃      0.7650  │     76.47 %  ┃   8.000e-03  │   02:15 min  ┃      1.5209  │     60.76 %  ┃
┃           5  ┃      0.3226  │     90.08 %  ┃   1.600e-03  │   02:15 min  ┃      1.0542  │     77.52 %  ┃
┃           6  ┃      0.1579  │     95.97 %  ┃   1.600e-03  │   02:15 min  ┃      1.0695  │     78.64 %  ┃
┃           7  ┃      0.0934  │     98.24 %  ┃   1.600e-03  │   02:15 min  ┃      1.0774  │     79.12 %  ┃
┃           8  ┃      0.0597  │     99.10 %  ┃   1.600e-03  │   02:15 min  ┃      1.0859  │     79.00 %  ┃
┃           9  ┃      0.0410  │     9

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:523: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:195: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


┃           0  ┃      1.5441  │     53.11 %  ┃   8.000e-03  │   04:23 min  ┃      1.0780  │     65.80 %  ┃
┃           1  ┃      0.8253  │     74.16 %  ┃   8.000e-03  │   04:23 min  ┃      0.8945  │     71.58 %  ┃
┃           2  ┃      0.6297  │     81.46 %  ┃   8.000e-03  │   04:23 min  ┃      0.8495  │     72.54 %  ┃
┃           3  ┃      0.4951  │     86.70 %  ┃   8.000e-03  │   04:24 min  ┃      0.8444  │     73.00 %  ┃
┃           4  ┃      0.4016  │     90.06 %  ┃   8.000e-03  │   04:23 min  ┃      0.7919  │     74.80 %  ┃
┃           5  ┃      0.2425  │     95.46 %  ┃   1.600e-03  │   04:23 min  ┃      0.6043  │     81.66 %  ┃
┃           6  ┃      0.1845  │     97.27 %  ┃   1.600e-03  │   04:23 min  ┃      0.5921  │     82.04 %  ┃
┃           7  ┃      0.1537  │     98.08 %  ┃   1.600e-03  │   04:23 min  ┃      0.5892  │     82.14 %  ┃
┃           8  ┃      0.1313  │     98.63 %  ┃   1.600e-03  │   04:24 min  ┃      0.5898  │     81.92 %  ┃
┃           9  ┃      0.1146  │     9

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:523: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:195: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when 

┃           0  ┃      4.3199  │     10.03 %  ┃   8.000e-03  │   04:35 min  ┃      3.1865  │     21.62 %  ┃
┃           1  ┃      3.3953  │     30.10 %  ┃   8.000e-03  │   04:35 min  ┃      2.3967  │     35.34 %  ┃
┃           2  ┃      2.6067  │     41.29 %  ┃   8.000e-03  │   04:35 min  ┃      1.9076  │     44.32 %  ┃
┃           3  ┃      2.1317  │     48.48 %  ┃   8.000e-03  │   04:35 min  ┃      1.6465  │     49.18 %  ┃
┃           4  ┃      1.8390  │     53.30 %  ┃   8.000e-03  │   04:35 min  ┃      1.4922  │     53.46 %  ┃
┃           5  ┃      1.6829  │     56.55 %  ┃   1.600e-03  │   04:35 min  ┃      1.4496  │     55.14 %  ┃
┃           6  ┃      1.6380  │     57.89 %  ┃   1.600e-03  │   04:35 min  ┃      1.4263  │     55.78 %  ┃
┃           7  ┃      1.6022  │     58.69 %  ┃   1.600e-03  │   04:35 min  ┃      1.4063  │     56.40 %  ┃
┃           8  ┃      1.5693  │     59.22 %  ┃   1.600e-03  │   04:35 min  ┃      1.3883  │     56.92 %  ┃
┃           9  ┃      1.5454  │     5

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
target_model = torch.load('model0.pt').to(device)
shadow_model = torch.load('model1.pt').to(device)

FileNotFoundError: ignored

In [ ]:
shadow_train = torch.load('train1.pt')
shadow_test = torch.load('test1.pt')
validate_train = torch.load('train0.pt')
validate_test = torch.load('test0.pt')
salem_membership_inference(target_model, shadow_train, shadow_test, validate_train, validate_test)
yeom_attack(device, validate_train, validate_test, target_model)